In [8]:
import requests
from urllib.parse import quote
import base64
import time
from datetime import datetime

In [9]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    
    def _get_mac_from_page(self, pageName):
        
        try:

            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, pageName), headers=self._get_headers())
            content = response.content

            ini = 'hostList = new Array('
            fin = ');'
            
            # nuevo cod
            texto = str(content)[str(content).index(ini)+len(ini)+2:].replace('\\n', '').replace(', 0,0 ', '')
            texto = texto[:texto.index(fin)].replace('-', ':').lower()

            lista = [l.replace("\"", "").replace(" ", "") for l in texto.split(',')]
            nueva_lista = [lista[i:i+4] for i in range(0, len(lista), 4) if lista[i+1] in ['0', '1', '8']]

            return nueva_lista
        
        except:
            return "Error"
    
    def get_all_macs(self):
        page = '1'
        macs = []
        mac_stat = []

        while True:
            arr_in = self._get_mac_from_page("WlanStationRpm.htm?Page="+page)
            arr = [l[0] for l in arr_in]
            page = str(int(page) + 1)

            if set(arr).issubset(set(macs)):
                break
            else:
                macs.extend(arr)
                mac_stat.extend(arr_in)

        mac_stat.sort(key=lambda x:int(x[2]), reverse=True)
        macs = [m[0] for m in mac_stat]
        return mac_stat #macs #
    
    def estadisticas(self):
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        ini = 'wlanHostPara = new Array('
        ini1 = 'statistList = new Array('
        fin = ');'

        try:
            # numero de conectados
            nconx = len(self.get_all_macs())
            
            # enviados y recibidos
            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, 'StatusRpm.htm'), headers=tl._get_headers())
            content = response.content
            texto = str(content)[str(content).index(ini1)+len(ini1)+2:]
            statistList = texto[:texto.index(fin)].split(',')

            b_rec = statistList[0]
            b_env = statistList[1]

            return [timestamp, nconx, b_rec, b_env]
        except:
            return "Error"
        
    def restart_router(self):
        params = (('Reboot', 'Reboot'),)
        try:
            res = requests.get('http://%s/userRpm/SysRebootRpm.htm' % self._host,
                                headers=self._get_headers(), params=params)
        except:
            pass
    

In [10]:
# https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv
import os

load_dotenv()
tplink_id = os.getenv("TPLINK_ID")
tplink_pw = os.getenv("TPLINK_SECRET")

tl = TippiLink(tplink_id, tplink_pw, "192.168.66.32")


In [11]:
print(tl.estadisticas())

[1618395770.950189, 21, '221407730', ' 669791596']


In [7]:
tl.get_all_macs()

[['f8:1a:67:cf:ef:82', '8', '790416', '908017'],
 ['50:9e:a7:77:ff:0b', '1', '103433', '107175'],
 ['b8:ae:ed:44:29:7a', '1', '85315', '86354'],
 ['60:14:66:d0:6c:94', '1', '63322', '58840'],
 ['b8:ae:ed:e1:5f:3c', '1', '16762', '14493'],
 ['d4:72:26:01:92:cc', '1', '16533', '1972'],
 ['d4:c9:4b:a3:69:6e', '1', '12060', '1068'],
 ['20:2d:07:51:6d:aa', '1', '7112', '2401'],
 ['9c:a9:e4:f5:d4:89', '1', '4925', '4227'],
 ['b4:1c:30:fa:0b:33', '1', '2279', '1166'],
 ['c0:17:4d:a0:20:b5', '0', '1514', '1248'],
 ['94:27:90:5a:8b:e8', '1', '1054', '640'],
 ['ac:57:75:28:2a:bc', '1', '768', '175'],
 ['44:ad:19:04:46:7f', '1', '580', '177'],
 ['24:46:c8:f6:40:4a', '1', '519', '147'],
 ['f8:e0:79:0e:cd:4d', '1', '397', '25'],
 ['60:1d:91:48:b9:10', '1', '318', '46'],
 ['60:14:66:e3:9b:99', '1', '279', '220'],
 ['34:23:ba:72:64:ff', '1', '113', '91'],
 ['84:db:ac:73:67:bb', '1', '88', '50'],
 ['68:5a:cf:9d:15:fa', '1', '34', '19'],
 ['0c:ec:84:84:de:9b', '1', '11', '16'],
 ['64:88:ff:cf:38:9b', '

Current Status - the connected wireless station's running status, one of 
STA-AUTH 0 
STA-ASSOC 1 
AP-UP 8 

STA-JOINED / 
WPA / 
WPA-PSK / 
WPA2 / 
WPA2-PSK / 

AP-DOWN


10 ?